In [1]:
import requests
import time
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
import statistics
import sys

SERVER_URL = "http://localhost:8080"

session = requests.Session()
session_lock = threading.Lock()

def make_requests(num_requests, client_id=0):
    """
    Виконати задану кількість запитів до /inc
    Повертає час виконання
    """
    # Кожен потік створює власну сесію для уникнення конфліктів
    local_session = requests.Session()
    local_session.headers.update({'Connection': 'keep-alive'})
    
    start_time = time.time()
    success_count = 0
    error_count = 0
    
    for i in range(num_requests):
        try:
            response = local_session.get(f"{SERVER_URL}/inc", timeout=5)
            if response.status_code == 200:
                success_count += 1
            else:
                error_count += 1
                if error_count <= 5:  # Виводимо тільки перші помилки
                    print(f"\tClient {client_id}: Request {i} failed with status {response.status_code}")
        except Exception as e:
            error_count += 1
            if error_count <= 5:
                print(f"\tClient {client_id}: Request {i} error: {e}")
        
        # Прогрес кожні 1000 запитів
        if (i + 1) % 1000 == 0:
            elapsed = time.time() - start_time
            rate = (i + 1) / elapsed
            print(f"   Client {client_id}: {i+1}/{num_requests} ({rate:.1f} req/s)")
    
    elapsed_time = time.time() - start_time
    
    if error_count > 0:
        print(f"\tClient {client_id}: {error_count} errors out of {num_requests} requests")
    
    local_session.close()
    return elapsed_time

def get_counter_value():
    """Отримати поточне значення каунтера"""
    try:
        response = requests.get(f"{SERVER_URL}/count", timeout=10)
        return response.json()['count']
    except Exception as e:
        print(f"\tError getting counter: {e}")
        return None

def reset_counter():
    """Скинути каунтер"""
    try:
        response = requests.get(f"{SERVER_URL}/reset", timeout=10)
        print("\tCounter reset")
    except Exception as e:
        print(f"\tError resetting counter: {e}")

def get_server_mode():
    """Отримати режим роботи сервера"""
    try:
        response = requests.get(f"{SERVER_URL}/mode", timeout=10)
        return response.json()['mode']
    except Exception as e:
        print(f"\tError getting server mode: {e}")
        return "unknown"

def run_test(num_clients, requests_per_client):
    """
    Тест з заданою кількістю клієнтів
    
    Args:
        num_clients: кількість одночасних клієнтів
        requests_per_client: кількість запитів на кожного клієнта
    """
    total_requests = num_clients * requests_per_client
    
    print()
    print('='*70)
    print(f"Тест: {num_clients} клієнт(ів), по {requests_per_client:,} запитів кожен")
    print(f"\tОчікуване значення count: {total_requests:,}")
    print('='*70)
    
    reset_counter()
    time.sleep(0.5)
    
    initial_count = get_counter_value()
    print(f"\tПочаткове значення counter: {initial_count}")
    
    # Запуск клієнтів паралельно
    start_time = time.time()
    
    with ThreadPoolExecutor(max_workers=num_clients) as executor:
        futures = []
        print(f"\nЗапуск {num_clients} клієнт(ів)...")
        for client_id in range(num_clients):
            future = executor.submit(make_requests, requests_per_client, client_id)
            futures.append(future)
        
        # Збір результатів
        client_times = []
        completed = 0
        for future in as_completed(futures):
            elapsed = future.result()
            client_times.append(elapsed)
            completed += 1
            print(f"Client {completed}/{num_clients} завершено за {elapsed:.2f} сек")
    
    total_time = time.time() - start_time
    
    final_count = get_counter_value()
    
    # Розрахунок метрик
    throughput = total_requests / total_time
    avg_client_time = statistics.mean(client_times)
    
    print(f"\nРезультати:")
    print(f"\tФінальне значення count: {final_count:,}")
    print(f"\tОчікуване значення: {total_requests:,}")
    
    if final_count is not None:
        lost = total_requests - final_count
        print(f"\tРізниця (lost updates): {lost:,}")
        if lost > 0:
            print(f"Виявлено {lost} втрачених оновлень!")
    
    print(f"\tЗагальний час виконання: {total_time:.2f} сек")
    print(f"\t Середній час на клієнта: {avg_client_time:.2f} сек")
    print(f"\t Throughput: {throughput:.2f} запитів/сек")
    print('='*70, "\n")
    
    return {
        'num_clients': num_clients,
        'requests_per_client': requests_per_client,
        'total_requests': total_requests,
        'final_count': final_count,
        'total_time': total_time,
        'throughput': throughput,
        'lost_updates': total_requests - (final_count or 0)
    }


# Тестування в режимі БД (PostgreSQL)

In [2]:
tests = [
        (1, 10000),   # 1 клієнт, 10K запитів
        (2, 10000),   # 2 клієнти, по 10K запитів
        (5, 10000),   # 5 клієнтів, по 10K запитів
        (10, 10000),  # 10 клієнтів, по 10K запитів
    ]
  
print("="*70)
print("Web Counter - Тестування продуктивності",)
print("="*70)

try:
    mode = get_server_mode()
    print(f"Режим сервера: {mode.upper()}")
except Exception as e:
    print(f"Не вдалося підключитися до сервера на {SERVER_URL}")
    sys.exit()

results = []

Web Counter - Тестування продуктивності
Режим сервера: DATABASE


In [3]:
test = tests[0][0]
requests_per_client = tests[0][1]

result = run_test(test, requests_per_client)
results.append(result)


Тест: 1 клієнт(ів), по 10,000 запитів кожен
	Очікуване значення count: 10,000
	Counter reset
	Початкове значення counter: 0

Запуск 1 клієнт(ів)...
   Client 0: 1000/10000 (147.8 req/s)
   Client 0: 2000/10000 (177.3 req/s)
   Client 0: 3000/10000 (188.4 req/s)
   Client 0: 4000/10000 (160.6 req/s)
   Client 0: 5000/10000 (158.7 req/s)
   Client 0: 6000/10000 (164.1 req/s)
   Client 0: 7000/10000 (167.6 req/s)
   Client 0: 8000/10000 (171.3 req/s)
   Client 0: 9000/10000 (173.2 req/s)
   Client 0: 10000/10000 (173.7 req/s)
Client 1/1 завершено за 57.56 сек

Результати:
	Фінальне значення count: 10,000
	Очікуване значення: 10,000
	Різниця (lost updates): 0
	Загальний час виконання: 57.56 сек
	 Середній час на клієнта: 57.56 сек
	 Throughput: 173.73 запитів/сек



In [4]:
test = tests[1][0]
requests_per_client = tests[1][1]

result = run_test(test, requests_per_client)
results.append(result)


Тест: 2 клієнт(ів), по 10,000 запитів кожен
	Очікуване значення count: 20,000
	Counter reset
	Початкове значення counter: 0

Запуск 2 клієнт(ів)...
   Client 0: 1000/10000 (59.5 req/s)
   Client 1: 1000/10000 (59.4 req/s)
   Client 0: 2000/10000 (75.9 req/s)
   Client 1: 2000/10000 (75.9 req/s)
   Client 0: 3000/10000 (82.0 req/s)
   Client 1: 3000/10000 (81.9 req/s)
   Client 0: 4000/10000 (88.4 req/s)
   Client 1: 4000/10000 (88.3 req/s)
   Client 0: 5000/10000 (95.0 req/s)
   Client 1: 5000/10000 (95.0 req/s)
   Client 0: 6000/10000 (94.3 req/s)
   Client 1: 6000/10000 (94.3 req/s)
   Client 0: 7000/10000 (98.3 req/s)
   Client 1: 7000/10000 (98.2 req/s)
   Client 0: 8000/10000 (101.4 req/s)
   Client 1: 8000/10000 (101.4 req/s)
   Client 0: 9000/10000 (104.7 req/s)
   Client 1: 9000/10000 (104.6 req/s)
   Client 0: 10000/10000 (107.6 req/s)
Client 1/2 завершено за 92.92 сек
   Client 1: 10000/10000 (107.6 req/s)
Client 2/2 завершено за 92.93 сек

Результати:
	Фінальне значення cou

In [5]:
test = tests[2][0]
requests_per_client = tests[2][1]

result = run_test(test, requests_per_client)
results.append(result)


Тест: 5 клієнт(ів), по 10,000 запитів кожен
	Очікуване значення count: 50,000
	Counter reset
	Початкове значення counter: 0

Запуск 5 клієнт(ів)...
   Client 4: 1000/10000 (15.9 req/s)
   Client 1: 1000/10000 (15.8 req/s)
   Client 3: 1000/10000 (15.8 req/s)
   Client 0: 1000/10000 (15.6 req/s)
   Client 2: 1000/10000 (15.6 req/s)
   Client 4: 2000/10000 (17.2 req/s)
   Client 1: 2000/10000 (17.2 req/s)
   Client 3: 2000/10000 (17.1 req/s)
   Client 0: 2000/10000 (16.9 req/s)
   Client 2: 2000/10000 (16.9 req/s)
   Client 4: 3000/10000 (17.0 req/s)
   Client 1: 3000/10000 (16.9 req/s)
   Client 3: 3000/10000 (16.8 req/s)
   Client 0: 3000/10000 (16.7 req/s)
   Client 2: 3000/10000 (16.7 req/s)
   Client 4: 4000/10000 (16.9 req/s)
   Client 1: 4000/10000 (16.8 req/s)
   Client 3: 4000/10000 (16.7 req/s)
   Client 2: 4000/10000 (16.7 req/s)
   Client 0: 4000/10000 (16.6 req/s)
   Client 4: 5000/10000 (16.2 req/s)
   Client 1: 5000/10000 (16.1 req/s)
   Client 3: 5000/10000 (16.0 req/s)


In [6]:
test = tests[3][0]
requests_per_client = tests[3][1]

result = run_test(test, requests_per_client)
results.append(result)


Тест: 10 клієнт(ів), по 10,000 запитів кожен
	Очікуване значення count: 100,000
	Counter reset
	Початкове значення counter: 0

Запуск 10 клієнт(ів)...
	Client 9: Request 277 error: HTTPConnectionPool(host='localhost', port=8080): Read timed out. (read timeout=5)	Client 7: Request 279 error: HTTPConnectionPool(host='localhost', port=8080): Read timed out. (read timeout=5)
	Client 2: Request 286 error: HTTPConnectionPool(host='localhost', port=8080): Read timed out. (read timeout=5)
	Client 5: Request 270 error: HTTPConnectionPool(host='localhost', port=8080): Read timed out. (read timeout=5)

	Client 1: Request 291 error: HTTPConnectionPool(host='localhost', port=8080): Read timed out. (read timeout=5)
   Client 4: 1000/10000 (5.9 req/s)
   Client 0: 1000/10000 (5.9 req/s)   Client 3: 1000/10000 (5.9 req/s)

   Client 6: 1000/10000 (5.8 req/s)
   Client 8: 1000/10000 (5.7 req/s)
   Client 9: 1000/10000 (5.6 req/s)
   Client 1: 1000/10000 (5.6 req/s)
   Client 2: 1000/10000 (5.6 req/s)


In [7]:
print("\n" + "="*80)
print("ПІДСУМКОВА ТАБЛИЦЯ РЕЗУЛЬТАТІВ")
print("="*80)
print(f"{'Клієнти':<10} {'Всього':<12} {'Count':<12} {'Час (с)':<12} {'Throughput':<18} {'Lost':<10}")
print(f"{'':10} {'запитів':12} {'':12} {'':12} {'(req/s)':18} {'updates':10}")
print("-"*80)

for r in results:
    print(f"{r['num_clients']:<10} {r['total_requests']:<12,} {r['final_count']:<12,} "
            f"{r['total_time']:<12.2f} {r['throughput']:<18.2f} {r['lost_updates']:<10,}")

print("="*80)

print("\nАналіз:")
if results:      
    total_lost = sum(r['lost_updates'] for r in results)
    if total_lost == 0:
        print(f"БЕЗ ВТРАТ (0 lost updates)")
    else:
        print(f" !!! Виявлено {total_lost} втрачених оновлень !!!")



ПІДСУМКОВА ТАБЛИЦЯ РЕЗУЛЬТАТІВ
Клієнти    Всього       Count        Час (с)      Throughput         Lost      
           запитів                                (req/s)            updates   
--------------------------------------------------------------------------------
1          10,000       10,000       57.56        173.73             0         
2          20,000       20,000       92.93        215.21             0         
5          50,000       50,000       651.42       76.75              0         
10         100,000      100,000      1505.11      66.44              0         

Аналіз:
БЕЗ ВТРАТ (0 lost updates)


# Тестування в режимі пам'яті

In [27]:
results = []

test = tests[0][0]
requests_per_client = tests[0][1]

result = run_test(test, requests_per_client)
results.append(result)


Тест: 1 клієнт(ів), по 10,000 запитів кожен
	Очікуване значення count: 10,000
	Counter reset
	Початкове значення counter: 0

Запуск 1 клієнт(ів)...
   Client 0: 1000/10000 (290.6 req/s)
   Client 0: 2000/10000 (421.3 req/s)
   Client 0: 3000/10000 (488.5 req/s)
   Client 0: 4000/10000 (536.4 req/s)
   Client 0: 5000/10000 (569.4 req/s)
   Client 0: 6000/10000 (594.8 req/s)
   Client 0: 7000/10000 (613.6 req/s)
   Client 0: 8000/10000 (626.5 req/s)
   Client 0: 9000/10000 (634.3 req/s)
   Client 0: 10000/10000 (639.9 req/s)
Client 1/1 завершено за 15.63 сек

Результати:
	Фінальне значення count: 10,000
	Очікуване значення: 10,000
	Різниця (lost updates): 0
	Загальний час виконання: 15.63 сек
	 Середній час на клієнта: 15.63 сек
	 Throughput: 639.83 запитів/сек



In [28]:
test = tests[1][0]
requests_per_client = tests[1][1]

result = run_test(test, requests_per_client)
results.append(result)


Тест: 2 клієнт(ів), по 10,000 запитів кожен
	Очікуване значення count: 20,000
	Counter reset
	Початкове значення counter: 0

Запуск 2 клієнт(ів)...
   Client 1: 1000/10000 (253.9 req/s)
   Client 0: 1000/10000 (239.1 req/s)
   Client 1: 2000/10000 (329.9 req/s)
   Client 0: 2000/10000 (322.5 req/s)
   Client 1: 3000/10000 (363.1 req/s)
   Client 0: 3000/10000 (361.2 req/s)
   Client 0: 4000/10000 (377.4 req/s)
   Client 1: 4000/10000 (376.7 req/s)
   Client 0: 5000/10000 (386.6 req/s)
   Client 1: 5000/10000 (385.6 req/s)
   Client 0: 6000/10000 (401.5 req/s)
   Client 1: 6000/10000 (397.1 req/s)
   Client 0: 7000/10000 (413.3 req/s)
   Client 1: 7000/10000 (407.0 req/s)
   Client 0: 8000/10000 (419.5 req/s)
   Client 1: 8000/10000 (412.4 req/s)
   Client 0: 9000/10000 (426.1 req/s)
   Client 1: 9000/10000 (417.9 req/s)
   Client 0: 10000/10000 (432.8 req/s)
Client 1/2 завершено за 23.11 сек
   Client 1: 10000/10000 (426.3 req/s)
Client 2/2 завершено за 23.46 сек

Результати:
	Фінальн

In [29]:
test = tests[2][0]
requests_per_client = tests[2][1]

result = run_test(test, requests_per_client)
results.append(result)


Тест: 5 клієнт(ів), по 10,000 запитів кожен
	Очікуване значення count: 50,000
	Counter reset
	Початкове значення counter: 0

Запуск 5 клієнт(ів)...
   Client 0: 1000/10000 (153.8 req/s)
   Client 4: 1000/10000 (153.3 req/s)
   Client 1: 1000/10000 (152.9 req/s)
   Client 2: 1000/10000 (147.2 req/s)
   Client 3: 1000/10000 (137.2 req/s)
   Client 0: 2000/10000 (181.2 req/s)
   Client 4: 2000/10000 (180.9 req/s)
   Client 1: 2000/10000 (178.2 req/s)
   Client 2: 2000/10000 (174.3 req/s)
   Client 3: 2000/10000 (159.7 req/s)
   Client 0: 3000/10000 (192.9 req/s)
   Client 4: 3000/10000 (192.3 req/s)
   Client 1: 3000/10000 (189.9 req/s)
   Client 2: 3000/10000 (184.5 req/s)
   Client 3: 3000/10000 (168.6 req/s)
   Client 0: 4000/10000 (201.1 req/s)
   Client 4: 4000/10000 (199.2 req/s)
   Client 1: 4000/10000 (198.4 req/s)
   Client 2: 4000/10000 (192.7 req/s)
   Client 3: 4000/10000 (174.6 req/s)
   Client 0: 5000/10000 (204.6 req/s)
   Client 4: 5000/10000 (202.2 req/s)
   Client 1: 50

In [30]:
test = tests[3][0]
requests_per_client = tests[3][1]

result = run_test(test, requests_per_client)
results.append(result)


Тест: 10 клієнт(ів), по 10,000 запитів кожен
	Очікуване значення count: 100,000
	Counter reset
	Початкове значення counter: 0

Запуск 10 клієнт(ів)...
   Client 6: 1000/10000 (87.6 req/s)
   Client 3: 1000/10000 (86.2 req/s)
   Client 2: 1000/10000 (86.0 req/s)
   Client 7: 1000/10000 (85.3 req/s)
   Client 1: 1000/10000 (85.0 req/s)
   Client 4: 1000/10000 (84.5 req/s)
   Client 9: 1000/10000 (84.4 req/s)
   Client 5: 1000/10000 (84.2 req/s)
   Client 8: 1000/10000 (81.1 req/s)
   Client 0: 1000/10000 (80.5 req/s)
   Client 2: 2000/10000 (97.6 req/s)
   Client 6: 2000/10000 (97.5 req/s)
   Client 3: 2000/10000 (97.3 req/s)
   Client 7: 2000/10000 (95.9 req/s)
   Client 4: 2000/10000 (93.9 req/s)
   Client 5: 2000/10000 (91.2 req/s)
   Client 8: 2000/10000 (91.0 req/s)
   Client 9: 2000/10000 (90.2 req/s)
   Client 0: 2000/10000 (90.0 req/s)
   Client 1: 2000/10000 (89.7 req/s)
   Client 2: 3000/10000 (104.0 req/s)
   Client 6: 3000/10000 (103.1 req/s)
   Client 3: 3000/10000 (102.5 r

In [31]:
print("\n" + "="*80)
print("ПІДСУМКОВА ТАБЛИЦЯ РЕЗУЛЬТАТІВ")
print("="*80)
print(f"{'Клієнти':<10} {'Всього':<12} {'Count':<12} {'Час (с)':<12} {'Throughput':<18} {'Lost':<10}")
print(f"{'':10} {'запитів':12} {'':12} {'':12} {'(req/s)':18} {'updates':10}")
print("-"*80)

for r in results:
    print(f"{r['num_clients']:<10} {r['total_requests']:<12,} {r['final_count']:<12,} "
            f"{r['total_time']:<12.2f} {r['throughput']:<18.2f} {r['lost_updates']:<10,}")

print("="*80)

print("\nАналіз:")
if results:      
    total_lost = sum(r['lost_updates'] for r in results)
    if total_lost == 0:
        print(f"БЕЗ ВТРАТ (0 lost updates)")
    else:
        print(f" !!! Виявлено {total_lost} втрачених оновлень !!!")



ПІДСУМКОВА ТАБЛИЦЯ РЕЗУЛЬТАТІВ
Клієнти    Всього       Count        Час (с)      Throughput         Lost      
           запитів                                (req/s)            updates   
--------------------------------------------------------------------------------
1          10,000       10,000       15.63        639.83             0         
2          20,000       20,000       23.46        852.50             0         
5          50,000       50,000       49.99        1000.26            0         
10         100,000      100,000      98.11        1019.26            0         

Аналіз:
БЕЗ ВТРАТ (0 lost updates)


# Висновок

Порівняльний аналіз показав, що реалізація каунтера в оперативній пам'яті забезпечує значно вищу пропускну здатність, досягаючи піку в 1019.26 req/s при 10 одночасних клієнтах, тоді як використання бази даних PostgreSQL суттєво обмежує продуктивність до 66.44 req/s у аналогічному сценарії. Основною причиною такого розриву є накладні витрати на мережеву взаємодію з БД та блокування транзакцій, що при великій кількості клієнтів призводило до затримок.

Попри різницю у швидкості, обидва методи продемонстрували повну відсутність втрачених оновлень (0 lost updates), що підтверджує успішну реалізацію потокобезпечності та цілісності даних. Таким чином, вибір між пам'яттю та БД є компромісом між максимальною швидкістю обробки та гарантованою сталістю даних даних після перезавантаження сервера 